In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import fnmatch
from IPython.display import Markdown as md

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.set_option("display.max_colwidth", None)

In [3]:
def list_to_dic(lst):
    index = [i for i in range(len(lst))]
    zipped = list(zip(lst,index))
    ret = {zipped[i][0]: zipped[i][1] for i in range(len(zipped))}
    return ret

In [9]:
def process_data(world):
    basepath = '../../data/'
    datapath = '../../docs/data/'
    
    with os.scandir(basepath + world) as entries:
        for entry in entries:
            if entry.is_file():
                event_file = entry.name
                event_name = (os.path.splitext(entry.name)[0])

                event_df = pd.read_csv(basepath + world + event_file)
                event_df.drop(columns=['event'], inplace=True)
                
                unique_players = list(event_df['player'].unique())
                player_mapping = list_to_dic(unique_players)
                event_df['player'] = event_df['player'].map(player_mapping)
                unique_players = list(event_df['player'].unique())

                event_df.rename(columns={'location.x': 'x', 'location.y': 'y', 'location.z': 'z', 'worldTime': 'time'}, inplace=True)
                display(md(f'## {event_name}'))
                display(event_df)

                path = Path(datapath + world + event_name + '/')
                path.mkdir(exist_ok=True)

                if fnmatch.fnmatch(event_file, 'PlayerDeathEvent.csv'):
                    make_csv_small(event_df, unique_players, datapath + world + event_name + '/', event_name)
                else:
                    make_csv_large(event_df, unique_players, datapath + world + event_name + '/', event_name)
                

In [5]:
def make_csv_large(event_df, players, path, event_name):
    for player in players:
        df = event_df[event_df['player'] == player]

        df = df[df.index % 20 == 0]

        filename = path + str(player) + '_' + event_name + '.csv'
        df.to_csv(filename, index=False)

In [6]:
def make_csv_small(event_df, players, path, event_name):
    for player in players:
        df = event_df[event_df['player'] == player]

        filename = path + str(player) + '_' + event_name + '.csv'
        df.to_csv(filename, index=False)

In [7]:
# def make_csv_plots(event_df, players, path, event_name):
#     for player in players:
#         df = event_df[event_df.index % 20 == 0]

#     filename = path + 'plot_' + event_name + '.csv'
#     df.to_csv(filename, index=False)

# Overworld

In [10]:
process_data('overworld/')

## ProjectileHitEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,1116,68,3,0,world,3722339,NaN,NaN,NaN,ARROW,SHEEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1082,65,-16,0,world,3758096,NaN,NaN,NaN,ARROW,CREEPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1080,66,-18,0,world,3758145,NaN,NaN,NaN,ARROW,CREEPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1087,64,-17,0,world,3758048,NaN,NaN,NaN,ARROW,CREEPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1072,63,0,0,world,3768916,NaN,NaN,NaN,ARROW,CREEPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677,1925,63,-325,7,world,44485727,NaN,NaN,NaN,ARROW,DROWNED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2678,1924,63,-322,7,world,44485885,NaN,NaN,NaN,ARROW,DROWNED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2679,1668,72,-270,7,world,44491355,NaN,NaN,NaN,ARROW,CREEPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2680,1668,72,-270,7,world,44491390,NaN,NaN,NaN,ARROW,CREEPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerQuitEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,-80,67,2,0,world,197233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-80,67,2,0,world,218900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-86,67,8,1,world,335561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-81,67,2,2,world,401319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,346,30,16,3,world,532240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,-4376,61,1106,25,world,47565613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,-4377,60,1106,25,world,48096024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
713,1667,77,-280,30,world,49328268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
714,1665,77,-279,30,world,49350030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerBedLeaveEvent

...

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,-80,67,2,0,world,196990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-80,67,2,0,world,218612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-101,64,-22,1,world,328380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-91,66,-21,2,world,400466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-86,63,-32,3,world,485739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,-4375,61,1108,25,world,47456476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
722,-4376,61,1106,25,world,47701280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,1663,73,-278,31,world,49303057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,1667,77,-280,31,world,49328811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerShearEntityEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,-332,71,89,0,world,617048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-342,76,72,0,world,616949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-362,70,26,0,world,617613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-327,70,104,0,world,622465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-328,70,108,0,world,622500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,1644,71,-285,7,world,42803429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,1648,72,-284,7,world,42803472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,1649,72,-283,7,world,42803476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
759,1651,72,-283,7,world,42803484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## EnchantItemEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,909,73,37,0,world,2676924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,909,73,36,0,world,2694349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,910,72,39,0,world,2728895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,911,72,40,0,world,2969918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,911,72,40,0,world,2981764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1667,62,-281,7,world,42842469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,1667,62,-281,7,world,44384953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,1667,62,-282,7,world,44385799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,1667,62,-281,7,world,44387210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Nether

In [11]:
process_data('nether/')

## ProjectileHitEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,144,89,-17,0,world_nether,3624255,NaN,NaN,NaN,ARROW,GHAST,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-18,57,181,0,world_nether,3658190,NaN,NaN,NaN,ARROW,BLAZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-18,57,181,0,world_nether,3658245,NaN,NaN,NaN,ARROW,BLAZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-20,58,181,0,world_nether,3658278,NaN,NaN,NaN,ARROW,BLAZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-19,57,180,0,world_nether,3658330,NaN,NaN,NaN,ARROW,BLAZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,-439,34,106,4,world_nether,42442392,NaN,NaN,NaN,ARROW,MAGMA_CUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733,-439,34,106,4,world_nether,42442459,NaN,NaN,NaN,ARROW,MAGMA_CUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,-439,34,106,4,world_nether,42442493,NaN,NaN,NaN,ARROW,MAGMA_CUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,-439,34,106,4,world_nether,42442524,NaN,NaN,NaN,ARROW,MAGMA_CUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerQuitEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,-17,57,168,0,world_nether,3659396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,900,49,948,1,world_nether,3950221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,179,36,217,2,world_nether,6486524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-65,45,138,2,world_nether,7666359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-132,45,-35,3,world_nether,8147782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-281,62,24,4,world_nether,9302707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-307,128,-159,4,world_nether,10790009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-305,128,-167,3,world_nether,10838471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-305,128,-169,4,world_nether,10883670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-62,54,177,2,world_nether,11125685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerBedLeaveEvent

...

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,-17,57,168,0,world_nether,3660194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,948,57,948,1,world_nether,3870529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,900,49,948,1,world_nether,4716023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,179,36,217,2,world_nether,6489283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-65,45,138,2,world_nether,8454053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-281,62,24,3,world_nether,9302757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-132,45,-35,4,world_nether,9470921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-307,128,-159,3,world_nether,10790056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-305,128,-167,4,world_nether,10838521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-305,128,-169,3,world_nether,10883705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerShearEntityEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action


## EnchantItemEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action


# End

In [12]:
process_data('end/')

## ProjectileHitEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,89,57,1,0,world_the_end,4320286,NaN,NaN,NaN,ARROW,ENDER_CRYSTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,91,55,1,0,world_the_end,4320010,NaN,NaN,NaN,ARROW,ENDER_CRYSTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91,56,1,0,world_the_end,4320183,NaN,NaN,NaN,ARROW,ENDER_CRYSTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-11,64,15,0,world_the_end,4321602,NaN,NaN,NaN,ARROW,ENDER_CRYSTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-10,64,14,0,world_the_end,4321549,NaN,NaN,NaN,ARROW,ENDER_CRYSTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-11,64,9,0,world_the_end,4321723,NaN,NaN,NaN,ARROW,ENDER_CRYSTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-18,64,14,0,world_the_end,4321864,NaN,NaN,NaN,ARROW,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-19,64,14,0,world_the_end,4321892,NaN,NaN,NaN,ARROW,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,62,-25,0,world_the_end,4322519,NaN,NaN,NaN,ARROW,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,63,-23,0,world_the_end,4322572,NaN,NaN,NaN,ARROW,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerQuitEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,262,1,3,0,world_the_end,11571149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,258,1,1,0,world_the_end,13353938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,260,1,1,0,world_the_end,13418102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1757,58,1498,1,world_the_end,17986082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7253,57,728,1,world_the_end,18244925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,260,1,0,2,world_the_end,18355720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,263,1,-1,3,world_the_end,18359136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,263,1,-1,3,world_the_end,19315180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,276,10,-10,4,world_the_end,19859303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,263,1,0,2,world_the_end,21048972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerBedLeaveEvent

...

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,262,1,3,0,world_the_end,11575628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,258,1,1,0,world_the_end,13381812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,260,1,1,0,world_the_end,14484633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1757,58,1498,1,world_the_end,18191118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7253,57,728,1,world_the_end,18244926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,260,1,0,2,world_the_end,19006087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,263,1,-1,3,world_the_end,19314970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,263,1,-1,3,world_the_end,19478510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,263,1,0,2,world_the_end,21048994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,263,1,0,2,world_the_end,22473234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PlayerShearEntityEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action


## EnchantItemEvent

,x,y,z,player,worldName,time,block,damaged,cause,weapon,target,oldWorld,itemDrop,entity,hatched,contents,itemHeld,action
0,259,1,0,0,world_the_end,13352013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,259,1,0,0,world_the_end,13406186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,260,1,-2,0,world_the_end,13408292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,259,1,-1,0,world_the_end,13413730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,259,1,-2,0,world_the_end,13415826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,262,1,-1,0,world_the_end,40270730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455,260,1,-1,0,world_the_end,40271006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,260,1,-1,0,world_the_end,40271051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,260,1,-1,0,world_the_end,40271182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Plots

In [18]:
# import plotly.graph_objects as go
# import plotly.express as px

In [19]:
def get_data(world, event):
    event_file = '../../data/' + world + '/' + event + '.csv'
    event_df = pd.read_csv(event_file)
    display(event_df)
    
    # drop unnecessary columns
    event_df.drop(columns=['_id','event'], inplace=True)
    display(event_df)
    
    unique_players = list(event_df['player'].unique())
    print(unique_players)
    
    player_mapping = list_to_dic(unique_players)
    
    event_df['player'] = event_df['player'].map(player_mapping)
    
    unique_players = list(event_df['player'].unique())
    print(unique_players)
    
    event_df.rename(columns={'location.x': 'x', 'location.y': 'y', 'location.z': 'z', 'worldTime': 'time'}, inplace=True)
    
    return (event_df, unique_players)

def make_csv_plots(move, players, world, folder):
    for player in players:
        # df = move[move['player'] == player]
        # print('BEFORE: ' + str(len(df)))
        #drop all odd numbered rows
    #     drop_i = [i for i in list(range(len(df))) if i % 2 == 1]
    #     df.drop(drop_i, inplace=True)
        df = move[move.index % 20 == 0]
        # print('AFTER: ', str(len(df)))

    filepath = '../../docs/data/' + world + "/" + folder + "/"
    filename = filepath + 'plot' + '_' + folder + '.csv'
    df.to_csv(filename, index=False)
    # return df
    

In [21]:
(df, unique_players) = get_data('overworld', 'PlayerMoveEvent')
make_csv_plots(df, unique_players, 'overworld', 'move')

,_id,event,location.x,location.y,location.z,player,block,damaged,cause,weapon,target,itemDrop,entity,hatched,contents,itemHeld,action
0,ObjectId(61671d6b0bec634729e689ba),PlayerMoveEvent,-83,67,2,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ObjectId(61671d6b0bec634729e689bd),PlayerMoveEvent,-85,67,3,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ObjectId(61671d6b0bec634729e689c1),PlayerMoveEvent,-88,67,4,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ObjectId(61671d6b0bec634729e689c4),PlayerMoveEvent,-90,67,5,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ObjectId(61671d6b0bec634729e689b8),PlayerMoveEvent,-81,67,2,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080762,ObjectId(618ace48e367df300459bde3),PlayerMoveEvent,-4374,61,1109,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080763,ObjectId(618ace48e367df300459bde4),PlayerMoveEvent,-4375,61,1109,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080764,ObjectId(618ace48e367df300459bde5),PlayerMoveEvent,-4375,61,1108,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080765,ObjectId(618ace48e367df300459bdeb),PlayerMoveEvent,-4375,61,1107,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,location.x,location.y,location.z,player,block,damaged,cause,weapon,target,itemDrop,entity,hatched,contents,itemHeld,action
0,-83,67,2,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-85,67,3,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-88,67,4,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-90,67,5,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-81,67,2,1d12324d5be14676989de8ac05a3ea9c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080762,-4374,61,1109,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080763,-4375,61,1109,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080764,-4375,61,1108,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080765,-4375,61,1107,3fb0a287002fc984a0f31204d2d5dcb1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['1d12324d5be14676989de8ac05a3ea9c', '9b667be1d5a97230ffc27443d4b57a7e', '848b613f77e69d3fe1d0fae912d67f22', '4ec2a2c48b7707339c2e090ed592f58c', '70cd1d9dae92efeb9dfbfdb1f5fd2c8a', 'cd1d675519ad2c32d140974f1a6f21a1', '7ae1ce09e166b11edb08be2c58017837', 'e2c31b3004c60ac1c0bd0f039d09ada5', 'd75e2d2c7578c76e4258ea588fbdec5a', '9ac5500bb331e30c89794c0a81610146', '216c21ada7085577b900fa3e9b84cea7', '613f75e9fffd8dbca8a4983e98f084ee', '134bce0ff5a216d08ff6a074da019a42', '770b6ee2bf3fca8e3d3c5ccfe93f835a', 'a3f26c36d64a2c231fb249f55ba995fc', 'dbed2aa95cc50bbf009fa1aa695ee036', '88379520abd0ce833ebeb2ce47cdb0a2', '6e973a24dca653c6017e821e2702894d', '3138051a1b18e314e37dbc738ff6bec5', '13a85178f705410fee811a2c839f5217', '11e132c1e726185f5065676807a09469', '79a7c2c9b738a6bfd10ca39646d9ad91', 'e83b0ea353d21f4e1ff956b100c4a6f8', '48acec2ec632017120071a946f292f28', '9abecea7dcb679913f4c2798fc332dd0', '3fb0a287002fc984a0f31204d2d5dcb1', '19dcb14ef8ae1cb8a3b7901813a9dcff', '1aed04a4e05930bda8c427cfa9

# Qualtrics Pre-Survey

In [67]:
df_q = pd.read_csv("../../data/01-qualtrics.csv")
df_q = df_q.drop(columns=['StartDate',	'EndDate',	'Status',	'IPAddress',	'Progress',	'Duration (in seconds)',	'Finished',	'RecordedDate',	'ResponseId',	'RecipientLastName',	'RecipientFirstName',	'RecipientEmail',	'ExternalReference',	'LocationLatitude',	'LocationLongitude',	'DistributionChannel',	'UserLanguage', 'mc_dim_4_TEXT'])	

In [68]:
df_q["consent"] = df_q["consent"].dropna()

In [70]:
df_q = df_q.drop(index=[0, 1, 2, 3, 47, 48, 49, 50, 51, 52, 53, 54,55, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 90]).reset_index(drop=True)
df_q.to_csv("../../data/02-qualtrics.csv", index=False)

In [71]:
df_q = df_q.drop(columns=["username"])

In [72]:
df_q.to_csv("../../public/data/01-qualtrics.csv", index=False)